In [1]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model="gpt-4o",
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key
)

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Define a tool that gets the weather for a city.
async def get_weather(city: str) -> str:
    """Get the weather for a city."""
    return f"The weather in {city} is 72 degrees and Sunny."


# Create an assistant agent.
weather_agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    tools=[get_weather],
    system_message="Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition.
text_termination = TextMentionTermination("TERMINATE")

# Create a single-agent team.
single_agent_team = RoundRobinGroupChat([weather_agent], termination_condition=text_termination)

async def run_team() -> None:
    result = await single_agent_team.run(task="What is the weather in New York?")
    print(result)


# Use `asyncio.run(run_team())` when running in a script.
await run_team()


/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py:343: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=15), content=[FunctionCall(id='call_fQ58ePuPXOqYnVPqx6zoGLGG', arguments='{"city":"New York"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', call_id='call_fQ58ePuPXOqYnVPqx6zoGLGG')], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant', models_usage=None, content='The weather in New York is 72 degrees and Sunny.', type='ToolCallSummaryMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=107, completion_tokens=13), content='The weather in New York is 72 degrees and sunny.', type='TextMessage'), TextMessage(source='assistant', models_usage=Request

In [3]:
from autogen_agentchat.base import TaskResult


async def run_team_stream() -> None:
    async for message in single_agent_team.run_stream(task="What is the weather in New York?"):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)


# Use `asyncio.run(run_team_stream())` when running in a script.
await run_team_stream()



source='user' models_usage=None content='What is the weather in New York?' type='TextMessage'


source='assistant' models_usage=RequestUsage(prompt_tokens=147, completion_tokens=15) content=[FunctionCall(id='call_NEUgqoc3YHyzLBEhvm7OjWyH', arguments='{"city":"New York"}', name='get_weather')] type='ToolCallRequestEvent'
source='assistant' models_usage=None content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', call_id='call_NEUgqoc3YHyzLBEhvm7OjWyH')] type='ToolCallExecutionEvent'
source='assistant' models_usage=None content='The weather in New York is 72 degrees and Sunny.' type='ToolCallSummaryMessage'
source='assistant' models_usage=RequestUsage(prompt_tokens=184, completion_tokens=13) content='The weather in New York is 72 degrees and sunny.' type='TextMessage'
source='assistant' models_usage=RequestUsage(prompt_tokens=202, completion_tokens=4) content='TERMINATE' type='TextMessage'
Stop Reason: Text 'TERMINATE' mentioned


In [4]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(single_agent_team.reset())` when running in a script.
await single_agent_team.reset()  # Reset the team for the next run.
# Use `asyncio.run(single_agent_team.run_stream(task="What is the weather in Seattle?"))` when running in a script.
await Console(
    single_agent_team.run_stream(task="What is the weather in Seattle?")
)  # Stream the messages to the console.

---------- user ----------
What is the weather in Seattle?


---------- assistant ----------
[FunctionCall(id='call_e25jHr0mTZ0rkUI7mjlgd0tm', arguments='{"city":"Seattle"}', name='get_weather')]
[Prompt tokens: 69, Completion tokens: 14]
---------- assistant ----------
[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', call_id='call_e25jHr0mTZ0rkUI7mjlgd0tm')]
---------- assistant ----------
The weather in Seattle is 72 degrees and Sunny.
---------- assistant ----------
The weather in Seattle is 72 degrees and Sunny. TERMPINATE
[Prompt tokens: 104, Completion tokens: 16]
---------- assistant ----------
TERMINATE
[Prompt tokens: 125, Completion tokens: 4]
---------- Summary ----------
Number of messages: 6
Finish reason: Text 'TERMINATE' mentioned
Total prompt tokens: 298
Total completion tokens: 34
Duration: 1.51 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in Seattle?', type='TextMessage'), ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=69, completion_tokens=14), content=[FunctionCall(id='call_e25jHr0mTZ0rkUI7mjlgd0tm', arguments='{"city":"Seattle"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', call_id='call_e25jHr0mTZ0rkUI7mjlgd0tm')], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant', models_usage=None, content='The weather in Seattle is 72 degrees and Sunny.', type='ToolCallSummaryMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=104, completion_tokens=16), content='The weather in Seattle is 72 degrees and Sunny. TERMPINATE', type='TextMessage'), TextMessage(source='assistant', models_usage=R

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(5)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Create a team with the primary and critic agents.
reflection_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=termination)

In [6]:
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write a short poem about fall season.")))` when running in a script.
await Console(
    reflection_team.run_stream(task="Write a short poem about fall season.")
)  # Stream the messages to the console.

---------- user ----------
Write a short poem about fall season.


---------- primary ----------
In autumn's golden light, the leaves cascade,
A dance of amber, russet, crimson played.
The whispering winds compose a gentle song,
As boughs adorned in splendor sway along.

The pumpkins gather in their orange glow,
Harvest moons that softly ebb and flow.
Cool air embraces with a crisp embrace,
While nature dons her warm and cozy grace.

Through fields of gold and forests' fiery lanes,
The beauty of the season still remains.
A time of rest, reflection, quiet cheer,
In autumn's breath, we find the year endear.
[Prompt tokens: 27, Completion tokens: 114]
---------- critic ----------
Your poem captures the essence of autumn beautifully with its vivid imagery and rhythmic flow. Here are a few suggestions for enhancing it further:

1. In the second line, consider rephrasing "A dance of amber, russet, crimson played" to maintain an active voice and avoid repetition of "played" in the past tense. Perhaps "A dance of amber, russet, crimson swayed."
   
2. In the 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=114), content="In autumn's golden light, the leaves cascade,\nA dance of amber, russet, crimson played.\nThe whispering winds compose a gentle song,\nAs boughs adorned in splendor sway along.\n\nThe pumpkins gather in their orange glow,\nHarvest moons that softly ebb and flow.\nCool air embraces with a crisp embrace,\nWhile nature dons her warm and cozy grace.\n\nThrough fields of gold and forests' fiery lanes,\nThe beauty of the season still remains.\nA time of rest, reflection, quiet cheer,\nIn autumn's breath, we find the year endear.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=159, completion_tokens=227), content='Your poem captures the essence of autumn beautifully with its vivid imagery and rhythmic flow. He

In [7]:
await Console(reflection_team.run_stream(task="중국 당나라 시풍으로 시를 써보세요."))

---------- user ----------
중국 당나라 시풍으로 시를 써보세요.


---------- primary ----------
금풍이 고운 잎을 물들이니 (金風染綺葉, 금풍염기엽)
가을 산은 단풍으로 붉어지네 (秋山變紅葉, 추산변홍엽)
달빛은 호수 위를 은처럼 비추고 (月光如銀湖, 월광여은호)
고요한 밤엔 갈대가 울려 퍼지네 (靜夜蘆葦泣, 정야노위읍)

길게 뻗은 산길엔 안개가 흐르고 (長路煙霞繞, 장로연하요)
물 흐르는 계곡엔 고요함이 깃드네 (潺湲靜谷幽, 잔원정곡유)
마음 한켠엔 시름이 남았지만 (心間憂思留, 심간우사유)
자연의 품속에서 평화를 찾으리 (自然懷安息, 자연회안식)
[Prompt tokens: 688, Completion tokens: 216]
---------- critic ----------
당나라 시풍을 잘 반영한 시입니다. 표현이 아름답고, 자연의 경치를 잘 묘사하고 있습니다. 몇 가지 추가적인 피드백을 드리자면:

1. "月光如銀湖" 라는 표현 대신 "銀湖月光映" 같은 어순을 고려해보는 것도 좋겠습니다.
2. "마음 한켠엔 시름이 남았지만" 부분을 좀 더 시적으로 바꾸면 좋겠습니다. 예를 들어, "마음 속 괴로움 있지만" 같은 느낌으로요.

이 피드백이 시를 더 풍부하게 만드는데 도움이 되었으면 합니다. 수정된 시를 다시 보여주시면 좋겠습니다.
[Prompt tokens: 922, Completion tokens: 151]
---------- primary ----------
고맙습니다. 주신 피드백을 반영하여 시를 수정해 보겠습니다.

금풍이 고운 잎을 물들이니 (金風染綺葉, 금풍염기엽)
가을 산은 단풍으로 붉어지네 (秋山變紅葉, 추산변홍엽)
달빛은 호수 위를 은처럼 비추고 (銀湖月光映, 은호월광영)
고요한 밤엔 갈대가 울려 퍼지네 (靜夜蘆葦泣, 정야노위읍)

길게 뻗은 산길엔 안개가 흐르고 (長路煙霞繞, 장로연하요)
물 흐르는 계곡엔 고요함이 깃드네 (潺湲靜谷幽, 잔원정곡유)
마음 속 괴로움 있지만 (心中愁思有, 심중수사유)
자연의 품속에서 평화를 찾으리 (自然懷安息, 자연회안식)

다시 한 번 감상해 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='중국 당나라 시풍으로 시를 써보세요.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=688, completion_tokens=216), content='금풍이 고운 잎을 물들이니 (金風染綺葉, 금풍염기엽)\n가을 산은 단풍으로 붉어지네 (秋山變紅葉, 추산변홍엽)\n달빛은 호수 위를 은처럼 비추고 (月光如銀湖, 월광여은호)\n고요한 밤엔 갈대가 울려 퍼지네 (靜夜蘆葦泣, 정야노위읍)\n\n길게 뻗은 산길엔 안개가 흐르고 (長路煙霞繞, 장로연하요)\n물 흐르는 계곡엔 고요함이 깃드네 (潺湲靜谷幽, 잔원정곡유)\n마음 한켠엔 시름이 남았지만 (心間憂思留, 심간우사유)\n자연의 품속에서 평화를 찾으리 (自然懷安息, 자연회안식)', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=922, completion_tokens=151), content='당나라 시풍을 잘 반영한 시입니다. 표현이 아름답고, 자연의 경치를 잘 묘사하고 있습니다. 몇 가지 추가적인 피드백을 드리자면:\n\n1. "月光如銀湖" 라는 표현 대신 "銀湖月光映" 같은 어순을 고려해보는 것도 좋겠습니다.\n2. "마음 한켠엔 시름이 남았지만" 부분을 좀 더 시적으로 바꾸면 좋겠습니다. 예를 들어, "마음 속 괴로움 있지만" 같은 느낌으로요.\n\n이 피드백이 시를 더 풍부하게 만드는데 도움이 되었으면 합니다. 수정된 시를 다시 보여주시면 좋겠습니다.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=1067, completion_

In [8]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="Always transfer to user when you don't know the answer. Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
termination = handoff_termination | text_termination

# Create a single-agent team.
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=termination)

In [9]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(Console(lazy_agent_team.run_stream(task="What is the weather in New York?")))` when running in a script.
await Console(lazy_agent_team.run_stream(task="What is the weather in New York?"))

---------- user ----------
What is the weather in New York?


---------- lazy_assistant ----------
[FunctionCall(id='call_kPPzKX25swcziVWVdP4jUa7r', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 68, Completion tokens: 11]
---------- lazy_assistant ----------
[FunctionExecutionResult(content='Transfer to user.', call_id='call_kPPzKX25swcziVWVdP4jUa7r')]
---------- lazy_assistant ----------
Transfer to user.
---------- Summary ----------
Number of messages: 4
Finish reason: Handoff to user from lazy_assistant detected.
Total prompt tokens: 68
Total completion tokens: 11
Duration: 0.49 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=68, completion_tokens=11), content=[FunctionCall(id='call_kPPzKX25swcziVWVdP4jUa7r', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='lazy_assistant', models_usage=None, content=[FunctionExecutionResult(content='Transfer to user.', call_id='call_kPPzKX25swcziVWVdP4jUa7r')], type='ToolCallExecutionEvent'), HandoffMessage(source='lazy_assistant', models_usage=None, target='user', content='Transfer to user.', type='HandoffMessage')], stop_reason='Handoff to user from lazy_assistant detected.')